In [15]:
import skopt
import numpy as np
import pandas as pd
import os
import math
import subprocess
from skopt import Optimizer
from skopt.space import Real
from skopt.learning.gaussian_process.kernels import ConstantKernel, Matern, RBF
from skopt.learning import GaussianProcessRegressor

## Test Functions

In [16]:
def hump_nf(args):
    x, y = args
    result = (4 - 2.1*x**2 + x**4/3)*x**2 + x*y + (-4 + 4*y**2)*y**2
    result = float(result)
    return result

In [17]:
def branin_nf(args):
    x, y = args
    result = np.square(y - (5.1/(4*np.square(math.pi)))*np.square(x) + 
         (5/math.pi)*x - 6) + 10*(1-(1./(8*math.pi)))*np.cos(x) + 10
    result = float(result)
    return result

In [18]:
def griewank_nf(args):
    x = np.array(args)
    result = 1 + 1.0/4000 * (x**2).sum() - np.prod(np.cos(x/np.arange(1, x.size + 1)**0.5))
    result = float(result)
    return result

In [19]:
def rosenbrock_nf(args):
    x, y = args
    result = 100*(y - x**2)**2 + (x - 1)**2
    result = float(result)
    return result

In [20]:
def easom_nf(args):
    x, y = args
    result = -np.cos(x)*np.cos(y)*np.exp(-(x-np.pi)**2-(y-np.pi)**2)
    result = float(result)
    return result

In [21]:
def colville(args):
    x1, x2, x3, x4 = args
    result = 100*(x1**2 - x2)**2 + (x1-1)**2 + (x3-1)**2 \
             + 90*(x3**2-x4)**2 + 10.1*((x2-1)**2 + (x4-1)**2) \
             + 19.8*(x2-1)*(x4-1)
    return result
    

## TEST ADR WITH EI

In [22]:
acq = 'EI'
n_calls = 40
noise_level = 0.0

In [23]:
# function, space to optimize
params = [(hump_nf,           [Real(-3, 3, name='x', transform='identity'),
                               Real(-2, 2, name='y', transform='identity')],
                              (np.array([0.0898, -0.7126], dtype=np.float),
                               np.array([-0.0898, 0.7126], dtype=np.float)), ('x', 'y')),
          (branin_nf,         [Real(-5, 10, name='x', transform='identity'),
                               Real(0, 15, name='y', transform='identity')],
                              (np.array([-np.pi, 12.275], dtype=np.float),
                               np.array([np.pi, 2.275], dtype=np.float),
                               np.array([9.42478, 2.475], dtype=np.float)), ('x', 'y')),
          (rosenbrock_nf,     [Real(-5, 10, name='x', transform='identity'),
                               Real(-5, 10, name='y', transform='identity')],
                              (np.array([1, 1], dtype=np.float),), ('x', 'y')),
          (griewank_nf,       [Real(-2, 2, name='x1', transform='identity'),
                               Real(-2, 2, name='x2', transform='identity'),
                               Real(-2, 2, name='x3', transform='identity'),
                               Real(-2, 2, name='x4', transform='identity'),
                               Real(-2, 2, name='x5', transform='identity'),
                               Real(-2, 2, name='x6', transform='identity')],
                              (np.zeros(6, dtype=np.float,)), ('x1', 'x2', 'x3', 'x4', 'x5', 'x6')),
          (easom_nf,          [Real(-100, 100, name='x', transform='identity'),
                               Real(-100, 100, name='y', transform='identity')],
                              (np.array([np.pi, np.pi], dtype=np.float),), ('x', 'y')),
          (colville,          [Real(-10, 10, name='x1', transform='identity'),
                               Real(-10, 10, name='x2', transform='identity'),
                               Real(-10, 10, name='x3', transform='identity'),
                               Real(-10, 10, name='x4', transform='identity')],
                              (np.array([1, 1, 1, 1], dtype=np.float),),
                              ('x1', 'x2', 'x3', 'x4'))
         ]

In [31]:
out_dir = os.path.join('results_ADR')
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

prev_func = None
kernels = [1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)),
           1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0), nu=2.5),
           1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0), nu=1.5),
           1.0 * RBF(length_scale=1.0, length_scale_bounds='fixed'),
           1.0 * Matern(length_scale=1.0, length_scale_bounds='fixed', nu=2.5),
           1.0 * Matern(length_scale=1.0, length_scale_bounds='fixed', nu=1.5)]
ker_names = ['RBF_UPD', 'MATERN5_UPD', 'MATERN3_UPD',
            'RBF_FIX', 'MATERN5_FIX', 'MATERN3_FIX']

for kernel, ker_name in zip(kernels, ker_names):
    for i, (func, space, pts, h_names) in enumerate(params):
        gpr = GaussianProcessRegressor(kernel=kernel, alpha=noise_level**2,
                                       normalize_y=True, noise="gaussian")
        opt_gp = Optimizer(space, base_estimator=gpr, acq_func=acq, n_initial_points=1,
                acq_optimizer="auto", random_state=np.random.RandomState(1))

        for n in range(n_calls):
            next_x = opt_gp.ask()
            next_y = func(next_x)
            result = opt_gp.tell(next_x, next_y)
            
        pure_loss = [func(x) for x in result.x_iters]
        dist = []
        for x in result.x_iters:
            min_dst = np.linalg.norm(x - pts[0])
            for k in range(1, len(pts)):
                min_dst = min(min_dst, np.linalg.norm(x - pts[k]))
            dist.append(min_dst)
        X = np.array(result.x_iters)
        df = pd.DataFrame({'loss': result.func_vals, 'pure_loss': pure_loss, 'dist': dist,
                           'iteration': np.arange(len(result.x_iters)),
                           **{name: X[:, i] for i, name in enumerate(h_names)}, 'kernel': ker_name})
        df = df.set_index('iteration')
        df.to_csv(os.path.join(out_dir, func.__name__ + '_' + ker_name)+'.csv')

/home/crus/anaconda3/envs/torch/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/crus/anaconda3/envs/torch/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
